## Bernoulli Programming Assignment
#### Enrico Absin
3. From the given code, modify the EM algorithm to become a Stochastic EM Algorithm.
4. Use the data from the paper: https://www.sciencedirect.com/science/article/abs/pii/S0031320322001753
5. Perform categorical clustering using the Bernoulli Mixture Model with Stochastic EM Algorithm.
6. Compare its performance with K-Modes Algorithm using Folkes-Mallows Index, Adjusted Rand Index, and Normalized Mutual Information Score.
7. Compare and contrast the performances, and explain what is happening (i.e. why is FMI always higher than ARI and NMI? Why is ARI and NMI low compared to FMI? etc.)
8. Write the report in Latex, push to your github with the codes.

In [14]:
import numpy as np
import pandas as pd
from kmodes.kmodes import KModes
from sklearn.metrics import fowlkes_mallows_score, adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics import fowlkes_mallows_score
from scipy.special import logsumexp

class StochasticBernoulliMixture:
    
    def __init__(self, n_components, max_iter, batch_size=100, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.tol = tol
    
    def fit(self, x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            
            for batch in self.iterate_batches():
                self.gamma = self.get_responsibilities(self.get_log_bernoullis(batch))
                self.remember_params()
                self.get_Neff()
                self.get_mu(batch)
                self.get_pi()
                
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.isnan(self.logL):
                self.reset_params()
                print(self.logL)
                break

    def iterate_batches(self):
        n_samples = len(self.x)
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        for start_idx in range(0, n_samples, self.batch_size):
            end_idx = min(start_idx + self.batch_size, n_samples)
            batch_indices = indices[start_idx:end_idx]
            yield self.x.iloc[batch_indices]

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)
        
    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()
    
    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()
        self.old_mu = None
        self.old_pi = None
        self.old_gamma = None
    
    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]
            
    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma
        
    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis
    
    def get_save_single(self, x, mu):
        epsilon = 1e-15
        mu_place = np.clip(mu, epsilon, 1 - epsilon)
        return np.tensordot(x, np.log(mu_place), (1,1))

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)
    
    def get_mu(self, batch):
        self.mu = np.einsum('ik,id -> kd', self.gamma, batch) / self.Neff[:, None]
        
    def get_pi(self):
        self.pi = self.Neff / self.n_samples
    
    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)
        
    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
    
    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))
        
    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)
    
    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

In [13]:
zoo_data = pd.read_csv("zoo.data", header=None)

X = zoo_data.iloc[:, 1:-1]
y = zoo_data.iloc[:, -1]

model_stochastic = StochasticBernoulliMixture(n_components=7, max_iter=100)
model_stochastic.fit(X)

model_kmodes = KModes(n_clusters=7, init='Huang', n_init=5, verbose=1)
cluster_labels_kmodes = model_kmodes.fit_predict(X)

fmi_stochastic = fowlkes_mallows_score(y, model_stochastic.predict(X))
ari_stochastic = adjusted_rand_score(y, model_stochastic.predict(X))
nmi_stochastic = normalized_mutual_info_score(y, model_stochastic.predict(X))

fmi_kmodes = fowlkes_mallows_score(y, cluster_labels_kmodes)
ari_kmodes = adjusted_rand_score(y, cluster_labels_kmodes)
nmi_kmodes = normalized_mutual_info_score(y, cluster_labels_kmodes)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 18, cost: 149.0
Run 1, iteration: 2/100, moves: 0, cost: 149.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 24, cost: 157.0
Run 2, iteration: 2/100, moves: 5, cost: 153.0
Run 2, iteration: 3/100, moves: 3, cost: 150.0
Run 2, iteration: 4/100, moves: 0, cost: 150.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 25, cost: 155.0
Run 3, iteration: 2/100, moves: 8, cost: 153.0
Run 3, iteration: 3/100, moves: 4, cost: 152.0
Run 3, iteration: 4/100, moves: 0, cost: 152.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 16, cost: 139.0
Run 4, iteration: 2/100, moves: 1, cost: 139.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 

In [12]:
print("Stochastic Bernoulli Mixture Model Performance:")
print("Folkes-Mallows Index (FMI):", fmi_stochastic)
print("Adjusted Rand Index (ARI):", ari_stochastic)
print("Normalized Mutual Information (NMI):", nmi_stochastic)

print("\nKModes Algorithm Performance:")
print("Folkes-Mallows Index (FMI):", fmi_kmodes)
print("Adjusted Rand Index (ARI):", ari_kmodes)
print("Normalized Mutual Information (NMI):", nmi_kmodes)

Stochastic Bernoulli Mixture Model Performance:
Folkes-Mallows Index (FMI): 0.48277252089435774
Adjusted Rand Index (ARI): 0.0
Normalized Mutual Information (NMI): 0.0

KModes Algorithm Performance:
Folkes-Mallows Index (FMI): 0.8766151924373514
Adjusted Rand Index (ARI): 0.840144831114853
Normalized Mutual Information (NMI): 0.8396495948660857


## Comparison Report
#### Several conclusions may be drawn from a comparison of the KModes algorithm's and the Stochastic Bernoulli Mixture Model's performances on the Zoo dataset using the Folkes-Mallows Index (FMI), Adjusted Rand Index (ARI), and Normalized Mutual Information (NMI) metrics. In many situations, the pairwise similarity between clusters and actual classes—the emphasis of the FMI—is larger than the ARI and NMI. This is due to the fact that FMI can produce a better score even in cases when there are obvious pairwise similarities but overall grouping is not flawless. On the other hand, if the clustering structure is not in good alignment with the true classes overall, then ARI, which quantifies overall clustering similarity, may be lower than FMI.In addition, if there is a great deal of uncertainty in the clustering assignments or if the clustering and class assignments are not closely related, NMI—which calculates mutual information and normalization—may be lower than FMI.
#### In particular, you can see which algorithm tends to have higher FMI values, whether ARI and NMI values are consistent between the two algorithms, and whether there are any situations where one algorithm performs better than the other based on these metrics when comparing the KModes algorithm and the Stochastic Bernoulli Mixture Model. These contrasts can shed light on the relative merits and shortcomings of each approach in terms of faithfully resolving the Zoo dataset's underlying clustering structure.